In [ ]:
import tensorflow as tf

import argparse
import numpy as np
import shlex
import subprocess
import sys
import wave
import json

from deepspeech import Model, printVersions
from timeit import default_timer as timer

try:
    from shhlex import quote
except ImportError:
    from pipes import quote

In [ ]:
! sudo apt-get install -y sox libsox-fmt-mp3

In [ ]:
#@title

! wget https://github.com/mozilla/DeepSpeech/releases/download/v0.6.1/deepspeech-0.6.1-models.tar.gz
! tar xvfz deepspeech-0.6.1-models.tar.gzb

In [ ]:
! ls ./

In [ ]:
ds = Model('./deepspeech-0.6.1-models/output_graph.pbmm', 500)

In [ ]:
desired_sample_rate = ds.sampleRate()

desired_sample_rate

In [ ]:
ds.enableDecoderWithLM('./deepspeech-0.6.1-models/lm.binary',
                       './deepspeech-0.6.1-models/trie',
                       0.75,
                       1.85)

In [ ]:
def convert_samplerate(audio_path, desired_sample_rate):
    sox_cmd = 'sox {} --type raw --bits 16 --channels 1 --rate {} --encoding signed-integer --endian little --compression 0.0 --no-dither - '.format(quote(audio_path), desired_sample_rate)
    try:
        output = subprocess.check_output(shlex.split(sox_cmd), stderr=subprocess.PIPE)
    except subprocess.CalledProcessError as e:
        raise RuntimeError('SoX returned non-zero status: {}'.format(e.stderr))
    except OSError as e:
        raise OSError(e.errno, 'SoX not found, use {}hz files or install it: {}'.format(desired_sample_rate, e.strerror))

    return desired_sample_rate, np.frombuffer(output, np.int16)

def load_audio_file(f):
    fin = wave.open(f, 'rb')
    fs = fin.getframerate()

    if fs != desired_sample_rate:
        print('Warning: original sample rate ({}) is different than {}hz. Resampling might produce erratic speech recognition.'.format(fs, desired_sample_rate), file=sys.stderr)
        fs, audio = convert_samplerate(f, desired_sample_rate)
    else:
        audio = np.frombuffer(fin.readframes(fin.getnframes()), np.int16)

    audio_length = fin.getnframes() * (1/fs)
    fin.close()

    return audio

def metadata_to_string(metadata):
    return ''.join(item.character for item in metadata.items)

def words_from_metadata(metadata):
    word = ""
    word_list = []
    word_start_time = 0
    # Loop through each character
    for i in range(0, metadata.num_items):
        item = metadata.items[i]
        # Append character to word if it's not a space
        if item.character != " ":
            word = word + item.character
        # Word boundary is either a space or the last character in the array
        if item.character == " " or i == metadata.num_items - 1:
            word_duration = item.start_time - word_start_time

            if word_duration < 0:
                word_duration = 0

            each_word = dict()
            each_word["word"] = word
            each_word["start_time "] = round(word_start_time, 4)
            each_word["duration"] = round(word_duration, 4)

            word_list.append(each_word)
            # Reset
            word = ""
            word_start_time = 0
        else:
            if len(word) == 1:
                # Log the start time of the new word
                word_start_time = item.start_time

    return word_list

def metadata_json_output(metadata):
    json_result = dict()
    json_result["words"] = words_from_metadata(metadata)
    json_result["confidence"] = metadata.confidence
    return json.dumps(json_result)

In [ ]:
AUDIO_DIR = './drive/My Drive/datasets/sounds/'

x = load_audio_file(AUDIO_DIR + 'female.wav')

In [ ]:
y = ds.stt(x)

In [ ]:
y.split(' ')

In [ ]:
print(metadata_to_string(ds.sttWithMetadata(x)))

In [ ]:
print(metadata_json_output(ds.sttWithMetadata(x)))